### File used to generate unforced trajectories, for comparison between learned and true dynamics

In [ ]:
import torch
import sys
import os
from matplotlib import pyplot as plt
import numpy as np
import matplotlib.cm as cm
import scipy
from datetime import datetime
import json

module_path = os.path.abspath(os.path.join('../..'))
if module_path not in sys.path:
	sys.path.insert(0, module_path)
print(sys.path)

import Double_Pendulum.Learning.autoencoders as autoencoders
import Double_Pendulum.robot_parameters as robot_parameters
import Double_Pendulum.transforms as transforms
import Double_Pendulum.dynamics as dynamics
import Plotting.pendulum_plot as pendulum_plot

from functools import partial



%load_ext autoreload
%autoreload 2


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

#### Starting parameters

In [ ]:
rp = robot_parameters.LUMPED_PARAMETERS.copy()
#rp["m0"] = 0.
plotter = pendulum_plot.Anim_plotter(rp)
neural_net = False
model_cw = False
model_shifting = True
print(rp["m0"])
print(rp)

armlen = rp["l0"] + rp["l1"]

q0_max = 0.

In [ ]:
t_start = 0.
t_end = 3.
dt = 0.01

model = autoencoders.Autoencoder_double(rp).to(device)
model_location = 'Models/NN_202505141642(half-q)/NN_202505141642_0.pth'
model.load_state_dict(torch.load(model_location, weights_only=True))

In [ ]:
def ik_fn(xy):
	return transforms.inverse_kinematics(xy, rp, model_cw).squeeze(0)

#### Generate $q$-space trajectory

In [ ]:
timestamp = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")

In [ ]:
def sim_q(t_start, t_end, dt, q_start, q_d_start, u):
	q = q_start
	q_d = q_d_start
	qs_ana = []
	q_ds_ana = []


	n_steps = int((t_end - t_start) / dt)

	for _ in range(n_steps):
		
		M_q, C_q, G_q = dynamics.dynamical_matrices(rp, q.squeeze(0), q_d.squeeze(0))
		A_q = dynamics.input_matrix(rp, q.squeeze(0))

		tau_q = A_q * u
		q_dd = (torch.pinverse(M_q) @ (tau_q - C_q @ q_d.T - G_q)).T
		q_d = q_d + q_dd * dt
		q = q + q_d * dt
		
		qs_ana.append(q.squeeze(0).detach())
		q_ds_ana.append(q_d.squeeze(0).detach())
	
	return torch.stack(qs_ana,  dim=0), torch.stack(q_ds_ana, dim=0)

In [ ]:
def sim_q_old(t_start, t_end, dt, q_start, q_d_start, u):
	q = q_start
	q_d = q_d_start

	q_ana_series, q_d_ana_series = torch.empty((0,2)).to(device), torch.empty((0,2)).to(device)

	n_steps = int((t_end - t_start) / dt)

	for t in torch.arange(t_start, t_end, dt):

		M_q, C_q, G_q = dynamics.dynamical_matrices(rp, q.squeeze(0), q_d.squeeze(0))
		A_q = dynamics.input_matrix(rp, q.squeeze(0))

		tau_q = A_q * u
		q_dd = (torch.pinverse(M_q) @ (tau_q - C_q @ q_d.T - G_q)).T
		q_d = q_d + q_dd * dt
		q = q + q_d * dt

		q_ana_series = torch.cat((q_ana_series, q.detach()), dim=0)
		q_d_ana_series = torch.cat((q_d_ana_series, q_d.detach()), dim=0)

	
	return q_ana_series, q_d_ana_series

#### Generate $\hat{q}$-space trajectory

In [ ]:
timestamp_alt = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")

q_nn = q_start
q_d_nn = q_d_start

th = model.encoder_vmap(q_nn)
th_d = (model.jacobian_enc(q_nn) @ q_d_nn.T).T

q_nn_series, q_d_nn_series = torch.empty((0,2)).to(device), torch.empty((0,2)).to(device)

for t in torch.arange(t_start, t_end, dt):

    is_clockwise = transforms.check_clockwise(q_nn.squeeze(0))

    if model_shifting:
            q_nn = transforms.shift_q(q_nn, clockwise=model_cw)



    q_hat = model.decoder_vmap(th, clockwise=model_cw)
    q_d_hat = (model.jacobian_dec(th) @ th_d.T).T

    M_q_hat, C_q_hat, G_q_hat = dynamics.dynamical_matrices(rp, q_hat.squeeze(0), q_d_hat.squeeze(0))

    J_h_inv = model.jacobian_dec(th, is_clockwise).squeeze(0)
    J_h_inv_trans = torch.transpose(J_h_inv, 0, 1)

    M_th, C_th, G_th = transforms.transform_dynamical_from_inverse(M_q_hat, C_q_hat, G_q_hat, th, th_d, J_h_inv, J_h_inv_trans)

    th_dd = (torch.pinverse(M_th) @ (-C_th @ th_d.T - G_th)).T
    th_d = th_d + th_dd * dt
    th = th + th_d * dt

    q_nn = model.decoder_vmap(th)
    q_d_nn = (model.jacobian_dec(th) @ th_d.T).T

    q_nn_series = torch.cat((q_nn_series, q_nn.detach()), dim = 0)
    q_d_nn_series = torch.cat((q_d_nn_series, q_d_nn.detach()), dim = 0)


In [ ]:
xy_ana_series = torch.empty((0,2)).to(device)
xy_nn_series = torch.empty((0,2)).to(device)

for q_ana, q_nn in zip(q_ana_series, q_nn_series):
	xy_ana, _ = transforms.forward_kinematics(rp, q_ana)
	xy_ana_series = torch.cat((xy_ana_series, xy_ana.unsqueeze(0)))
	xy_nn, _ = transforms.forward_kinematics(rp, q_nn)
	xy_nn_series = torch.cat((xy_nn_series, xy_nn.unsqueeze(0)))

In [ ]:
save_dir = f"Performance_Sims/simulation_{timestamp}"
os.makedirs(save_dir, exist_ok=True)

metadata = {
	"timestamp": timestamp,
	"Learned transform": False,
	"Model location": None,
	"Model clockwise": None,
	"Model shifting": None, 
	"xy start": {"x": xy_start[0].item(), "y": xy_start[1].item()},
	"Time step": dt,
	"Sim time": t_end,
}

metadata_path = os.path.join(save_dir, "metadata.json")
with open(metadata_path, "w") as f:
	json.dump(metadata, f, indent=4)

In [ ]:
stride = 3
pos_end_q_ana, pos_elbow_q_ana = torch.vmap(transforms.forward_kinematics, in_dims=(None, 0))(rp, q_ana_series[::stride])
pos_end_q_nn, pos_elbow_q_nn = torch.vmap(transforms.forward_kinematics, in_dims=(None, 0))(rp, q_nn_series[::stride])

frames_q_ana = plotter.frame_pendulum(pos_end_q_ana, pos_elbow_q_ana)
frames_q_nn = plotter.frame_pendulum(pos_end_q_nn, pos_elbow_q_nn)

data_ana = {
	"frames": frames_q_ana,
	"times": dt,
	"name": "q_ana", 
	"arm_color": "tab:blue",
	"act_color": "tab:cyan"
}

data_nn = {
	"frames": frames_q_nn,
	"times": dt,
	"name": "q_est",
	"arm_color": "tab:orange", 
	"act_color": "tab:red"
} 


frames_data = [data_ana, data_nn]

name_rp = "RP:(" + str(rp["xa"]) + "," + str(rp["ya"]) + ")_"


if neural_net:
	model_type = "NN"
else:
	model_type = "AL"

file_name = "t_end:[" + str(t_end) + "]_dt:[" + str(dt) + "]_stride:[" + str(stride) + "]_0.mp4"
file_counter = 0

output_path = os.path.join(save_dir, file_name)

while os.path.isfile(output_path):
	print("file name already exists")
	file_counter += 1
	file_name = file_name[:-6] + "_" + str(file_counter) + ".mp4"
	output_path = os.path.join(save_dir, file_name)

plotter.animate_pendulum(frames_data, ref_poss=None, plot_actuator=False, save_path=output_path, fps = 1/(dt*stride), dt = dt*stride)


In [ ]:
datasets_q = [
	{
	    "name": "nn",
	    "values": q_nn_series.cpu().detach().numpy(),
	    "color": "tab:orange"
	},
	{
		"name": "ana",
		"values": q_ana_series.cpu().detach().numpy(),
		"color": "tab:blue"
	}
]

datasets_xy = [
	{
	    "name": "nn",
	    "values": xy_nn_series.cpu().detach().numpy(),
	    "color": "tab:orange"
	},
	{
		"name": "ana",
		"values": xy_ana_series.cpu().detach().numpy(),
		"color": "tab:blue"
	}
]


# Common labels for the plots.
name_q = "q trajectory"
name_xy = "xy trajectory"
t_series = torch.arange(0, t_end, dt)

# Create an instance of ErrorPlotter.
ep = pendulum_plot.Error_plotter(rp)

# Prepare plot datasets for each column.
# Each call groups a set of datasets to be drawn in one subplot column.
q_dataset = ep.create_plot_dataset(t=t_series, datasets=datasets_q, reference=None, name=name_q)
xy_dataset = ep.create_plot_dataset(t=t_series, datasets=datasets_xy, reference=None, name=name_xy)


plot_datasets = [q_dataset, xy_dataset]
plot_colormaps = ["Oranges", "Blues", "Greens"]

file_name = "Error plot.png"
file_counter = 0

output_path = os.path.join(save_dir, file_name)

# Pass the list of columns (plot_dataset objects) to plot_multi.
#ep.plot_multi(plot_datasets=plot_datasets, save_path=output_path, axes_names = ["q", "th", "xy"])




#### Plot $q$-space error

In [ ]:
fig, ax = plt.subplots(figsize=(6,4))
for i, cmap in zip(range(2), plot_colormaps):
	t = q_dataset["x"]                      # shape (N,)
	q1 = q_dataset["data"][i]["y1"]         # shape (N,)
	q2 = q_dataset["data"][i]["y2"]         # shape (N,)
	sc = ax.scatter(q1, q2, c=t, cmap=cmap, s=20)

	# optionally connect points in order
	ax.plot(q1, q2, lw=0.5, color='gray', alpha=0.5)

# labels and colorbar
ax.set_xlabel('$q_0$')
ax.set_ylabel('$q_1$')
ax.grid()
#ax.set_xlim(-torch.pi, torch.pi)
#ax.set_ylim(-torch.pi, torch.pi)
cbar = plt.colorbar(sc, ax=ax)
cbar.set_label('time')

ax.set_title('Trajectory in $(q_0,q_1)$ colored by time')
plt.tight_layout()
plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(6,5))
for i, cmap in zip(range(2), plot_colormaps):
	t = xy_dataset["x"]                      # shape (N,)
	q1 = xy_dataset["data"][i]["y1"]         # shape (N,)
	q2 = xy_dataset["data"][i]["y2"]         # shape (N,)
	sc = ax.scatter(q1, q2, c=t, cmap=cmap, s=20)

	# optionally connect points in order
	ax.plot(q1, q2, lw=0.5, color='gray', alpha=0.5)

armlen = rp["l0"] + rp["l1"]
circ = plt.Circle((0, 0), armlen, color="gray", fill=False, linestyle="dashed", alpha=0.8)
ax.add_patch(circ)
# labels and colorbar
ax.set_xlabel('$x$')
ax.set_ylabel('$y$')
ax.grid()
ax.set_xlim(-armlen, armlen)
ax.set_ylim(-armlen, armlen)
cbar = plt.colorbar(sc, ax=ax)
cbar.set_label('time')

ax.set_title('Trajectory in $(x,y)$ colored by time')
plt.tight_layout()
plt.show()